## Topic Modeling–Latent Dirichlet Allocation

In [10]:
import warnings
warnings.filterwarnings('ignore')

/Users/sallypants/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pyLDAvis
import pyLDAvis.gensim_models

In [12]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [13]:
from textblob import TextBlob
from wordcloud import WordCloud
import spacy

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.feature_extraction import text
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora

In [16]:
df = pd.read_csv("combined_season1-37.tsv.zip", delimiter='\t')

In [17]:
df

,round,value,daily_double,category,comments,answer,question,air_date,notes
0,1,100,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-
1,1,200,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-
2,1,800,yes,LAKES & RIVERS,-,River in this famous song:,the Volga River,1984-09-10,-
3,1,400,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-
4,1,500,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-
...,...,...,...,...,...,...,...,...,...
389440,2,400,no,FOUNDRY,-,This hefty noisemaker from Whitechapel Foundry...,Big Ben,2021-08-13,-
389441,2,800,no,FOUNDRY,-,"Around 4,000 years ago, the first foundries in...",bronze,2021-08-13,-
389442,2,1200,no,FOUNDRY,-,Several different foundries worked for 4 month...,Monitor,2021-08-13,-
389443,2,1600,no,FOUNDRY,-,"Once one of the largest of its kind, the Gary ...",U.S. Steel,2021-08-13,-


In [18]:
df['air_date'] = pd.to_datetime(df['air_date'])

In [19]:
#On 11/26/2001, the values for the questions doubled for both rounds of Jeopardy. Need to adjust the earlier episodes to have the same values as post-11/26/2001 shows.
df.loc[df['air_date'] < '2001-11-26', "value"] = df.value * 2

In [20]:
#remove Daily Doubles since the contestants can wager any amounts for those
df = df[df["daily_double"] != 'yes']

In [21]:
#keep only standard values (this will remove Final Jeopardy questions, which do not have a set amount and set are at '0', as well as the handful of non-standard values that are likely typos)
df = df.loc[df['value'].isin([200, 400, 600, 800, 1000, 400, 800, 1200, 1600, 2000])]

In [22]:
#create a new column that contains questions, answers, and category
df["q_a_and_cat"] = df["answer"] + ' ' + df["question"] + ' ' + df["category"]

In [40]:
#tokenize words in "q_and_a" column
df['q_and_a'] = df['q_and_a'].apply(word_tokenize)

#lemmatize "q_and_a" column
lemmatizer = WordNetLemmatizer()
df['q_and_a'] = df['q_and_a'].apply(lambda row:[lemmatizer.lemmatize(word) for word in row])

df

,round,value,daily_double,category,comments,answer,question,air_date,notes,q_and_a
0,1,200,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-,"[River, mentioned, most, often, in, the, Bible..."
1,1,400,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-,"[Scottish, word, for, lake, loch]"
3,1,800,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-,"[American, river, only, 33, mile, shorter, tha..."
4,1,1000,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-,"[World, 's, largest, lake, ,, nearly, 5, time,..."
5,1,200,no,INVENTIONS,-,Marconi's wonderful wireless,the radio,1984-09-10,-,"[Marconi, 's, wonderful, wireless, the, radio]"
...,...,...,...,...,...,...,...,...,...,...
389438,2,1600,no,LOST,-,"In ""A Moveable Feast"", Gertrude Stein is quote...",Lost Generation,2021-08-13,-,"[In, ``, A, Moveable, Feast, '', ,, Gertrude, ..."
389440,2,400,no,FOUNDRY,-,This hefty noisemaker from Whitechapel Foundry...,Big Ben,2021-08-13,-,"[This, hefty, noisemaker, from, Whitechapel, F..."
389441,2,800,no,FOUNDRY,-,"Around 4,000 years ago, the first foundries in...",bronze,2021-08-13,-,"[Around, 4,000, year, ago, ,, the, first, foun..."
389442,2,1200,no,FOUNDRY,-,Several different foundries worked for 4 month...,Monitor,2021-08-13,-,"[Several, different, foundry, worked, for, 4, ..."


In [76]:
#change the column so it's not a list of lists
df['q_and_a'] = df['q_and_a'].str.join(" ")
df

,round,value,daily_double,category,comments,answer,question,air_date,notes,q_and_a
0,1,200,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-,R i v e r m e n t i o n e d m o s t o f ...
1,1,400,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-,S c o t t i s h w o r d f o r l a k e ...
3,1,800,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-,A m e r i c a n r i v e r o n l y 3 3 ...
4,1,1000,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-,"W o r l d ' s l a r g e s t l a k e , n ..."
5,1,200,no,INVENTIONS,-,Marconi's wonderful wireless,the radio,1984-09-10,-,M a r c o n i ' s w o n d e r f u l w i r ...
...,...,...,...,...,...,...,...,...,...,...
389438,2,1600,no,LOST,-,"In ""A Moveable Feast"", Gertrude Stein is quote...",Lost Generation,2021-08-13,-,"I n "" A M o v e a b l e F e a s t "" , ..."
389440,2,400,no,FOUNDRY,-,This hefty noisemaker from Whitechapel Foundry...,Big Ben,2021-08-13,-,T h i s h e f t y n o i s e m a k e r f ...
389441,2,800,no,FOUNDRY,-,"Around 4,000 years ago, the first foundries in...",bronze,2021-08-13,-,"A r o u n d 4 , 0 0 0 y e a r s a g o , ..."
389442,2,1200,no,FOUNDRY,-,Several different foundries worked for 4 month...,Monitor,2021-08-13,-,S e v e r a l d i f f e r e n t f o u n d ...


In [45]:
#remove stopwords
stop = stopwords.words('english')
df['q_and_a'] = df['q_and_a'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df

,round,value,daily_double,category,comments,answer,question,air_date,notes,q_and_a
0,1,200,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-,River mentioned often Bible Jordan
1,1,400,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-,Scottish word lake loch
3,1,800,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-,American river 33 mile shorter Mississippi Mis...
4,1,1000,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-,"World 's largest lake , nearly 5 time big Supe..."
5,1,200,no,INVENTIONS,-,Marconi's wonderful wireless,the radio,1984-09-10,-,Marconi 's wonderful wireless radio
...,...,...,...,...,...,...,...,...,...,...
389438,2,1600,no,LOST,-,"In ""A Moveable Feast"", Gertrude Stein is quote...",Lost Generation,2021-08-13,-,"In `` A Moveable Feast '' , Gertrude Stein quo..."
389440,2,400,no,FOUNDRY,-,This hefty noisemaker from Whitechapel Foundry...,Big Ben,2021-08-13,-,This hefty noisemaker Whitechapel Foundry bega...
389441,2,800,no,FOUNDRY,-,"Around 4,000 years ago, the first foundries in...",bronze,2021-08-13,-,"Around 4,000 year ago , first foundry Mesopota..."
389442,2,1200,no,FOUNDRY,-,Several different foundries worked for 4 month...,Monitor,2021-08-13,-,Several different foundry worked 4 month build...


## NLP Data Cleaning

In [46]:
# import string to remove punctuation

import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [47]:
def no_punctuation(text):
    nopunct=[words for words in text if words not in string.punctuation]
    words_without_punct=''.join(nopunct)
    return words_without_punct

In [48]:
#remove punctuation and lowercase words in 'category,' 'answer,' and 'question'

df['q_and_a'] = df['q_and_a'].apply(lambda x: no_punctuation(x).lower())

In [49]:
#remove numerals 

df['q_and_a'] = df['q_and_a'].str.replace('\d+', '')

### LDA with sklearn

In [21]:
#Use CountVectorizer to make a bag of words matrix, excluding stopwords, triwords
vectorizer = CountVectorizer(max_features=1000, stop_words='english', ngram_range= (1, 3))
vectorized_answer = vectorizer.fit_transform(df.answer)

In [22]:
vectorized_answer.shape

(363765, 1000)

In [26]:
#Run LDA
lda = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=54)
lda.fit_transform(vectorized_answer)

array([[0.03333333, 0.03333333, 0.03333333, ..., 0.36666665, 0.03333333,
        0.36666668],
       [0.025     , 0.025     , 0.525     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.27499606,
        0.52499812],
       ...,
       [0.03333333, 0.03333333, 0.03333333, ..., 0.03333333, 0.36666666,
        0.03333333],
       [0.27499999, 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.52499765]])

In [29]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))                        

    print()

In [30]:
print("\nTopics in LDA model:")
tf_feature_names = vectorizer.get_feature_names()

print_top_words(lda, tf_feature_names, 40)


Topics in LDA model:
Topic #0:
said letter people play hit dont way family know shows short death god head wife reports led girl leader blue law want includes takes refers thomas told citys books killed baby paul tom central animals hero drama money lord european
Topic #1:
new th war make life century york body took gave team including battle german father countrys general robert kids court military look things april real ship june run female bad daughter royal mans delivers square foot prince united civil yearold
Topic #2:
word clue country book song says company little use red musical number near right large help color car light late ones women building david heart win flag dance region piece does san mary sun western doesnt winner county bridge governor
Topic #3:
called crew years time king won im meaning latin set north museum founded story high feet university second night youll college art dog african oscar bc begins held event stands charles species going original islands field

### LDA with Gensim

In [29]:
df

,round,value,daily_double,category,comments,answer,question,air_date,notes,q_a_and_cat
0,1,200,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-,River mentioned most often in the Bible the Jo...
1,1,400,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-,Scottish word for lake loch LAKES & RIVERS
3,1,800,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-,American river only 33 miles shorter than the ...
4,1,1000,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-,"World's largest lake, nearly 5 times as big as..."
5,1,200,no,INVENTIONS,-,Marconi's wonderful wireless,the radio,1984-09-10,-,Marconi's wonderful wireless the radio INVENTIONS
...,...,...,...,...,...,...,...,...,...,...
389438,2,1600,no,LOST,-,"In ""A Moveable Feast"", Gertrude Stein is quote...",Lost Generation,2021-08-13,-,"In ""A Moveable Feast"", Gertrude Stein is quote..."
389440,2,400,no,FOUNDRY,-,This hefty noisemaker from Whitechapel Foundry...,Big Ben,2021-08-13,-,This hefty noisemaker from Whitechapel Foundry...
389441,2,800,no,FOUNDRY,-,"Around 4,000 years ago, the first foundries in...",bronze,2021-08-13,-,"Around 4,000 years ago, the first foundries in..."
389442,2,1200,no,FOUNDRY,-,Several different foundries worked for 4 month...,Monitor,2021-08-13,-,Several different foundries worked for 4 month...


In [24]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [25]:
#tokenize text and remove punctuation

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(df.q_a_and_cat))
print(data_words[:1])

[['river', 'mentioned', 'most', 'often', 'in', 'the', 'bible', 'the', 'jordan', 'lakes', 'rivers']]


In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['river', 'mentioned', 'most', 'often', 'in', 'the', 'bible', 'the', 'jordan', 'lakes_rivers']


In [27]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [31]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mention', 'often']]


In [32]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=40, 
                                           random_state=100,
                                           update_every=5,
                                           chunksize=1000,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
# Print the keyword of topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(16, '0.081*"would" + 0.041*"cut" + 0.037*"bone" + 0.030*"sweet" + 0.028*"service" + 0.027*"religious" + 0.026*"dress" + 0.024*"ballet" + 0.024*"trip" + 0.024*"ice"'), (35, '0.087*"foot" + 0.066*"kid" + 0.036*"wind" + 0.036*"discover" + 0.035*"building" + 0.035*"gold" + 0.033*"region" + 0.029*"folk" + 0.026*"hair" + 0.024*"depict"'), (31, '0.106*"small" + 0.056*"member" + 0.055*"green" + 0.047*"geography" + 0.046*"kind" + 0.040*"planet" + 0.029*"ride" + 0.026*"office" + 0.025*"bar" + 0.025*"nearly"'), (22, '0.064*"need" + 0.064*"think" + 0.045*"visit" + 0.039*"military" + 0.035*"hour" + 0.034*"greek" + 0.033*"class" + 0.028*"fast" + 0.027*"european" + 0.025*"material"'), (18, '0.081*"death" + 0.059*"classic" + 0.059*"friend" + 0.055*"category" + 0.036*"send" + 0.036*"house" + 0.033*"comedy" + 0.029*"deep" + 0.027*"direct" + 0.027*"newspaper"'), (9, '0.092*"back" + 0.077*"food" + 0.070*"second" + 0.055*"eat" + 0.044*"meet" + 0.035*"drink" + 0.034*"deal" + 0.029*"magazine" + 0.029*"drop

In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
30     0.046075  0.225926       1        1  5.557383
24     0.087148 -0.152406       2        1  3.981469
37     0.174330 -0.099571       3        1  3.713320
34     0.040743  0.070636       4        1  3.701454
27    -0.175424  0.023986       5        1  3.575084
39     0.180839 -0.150744       6        1  3.563335
20    -0.068507 -0.171159       7        1  3.383606
17    -0.087721  0.094926       8        1  3.254655
13     0.063005 -0.010788       9        1  3.053792
36    -0.190392 -0.124155      10        1  2.852750
1     -0.078059 -0.053016      11        1  2.694468
32     0.064284 -0.004655      12        1  2.685697
29    -0.091115 -0.068880      13        1  2.672755
5      0.050695 -0.064120      14        1  2.595801
6     -0.000076 -0.145170      15        1  2.551059
0      0.032984 -0.001043      16        1  2.504650
28     0.086979  0.063518      17        1  2.443587
8      0.078718  0.079517      18        1  2.391342
25     0.085390  0.013426      19        1  2.385985
7     -0.186102  0.018393      20        1  2.381201
23     0.002504  0.033574      21        1  2.362331
14    -0.015518  0.038747      22        1  2.324452
21    -0.043982  0.021332      23        1  2.177187
10     0.030773  0.039828      24        1  2.096302
2     -0.030602  0.087320      25        1  2.065773
26     0.067221  0.058092      26        1  2.037620
15    -0.022687  0.015023      27        1  2.026194
38    -0.009978  0.045524      28        1  1.984975
4      0.041960 -0.030014      29        1  1.979031
3     -0.049997 -0.074942      30        1  1.970452
11     0.036690  0.046340      31        1  1.934103
33    -0.081219 -0.075926      32        1  1.916289
12    -0.060317  0.022921      33        1  1.893343
19     0.024973  0.031870      34        1  1.881292
9      0.047270  0.021898      35        1  1.727421
18     0.041440 -0.005251      36        1  1.619441
22    -0.046232  0.060505      37        1  1.614673
31     0.017272  0.054088      38        1  1.517143
35    -0.060064  0.045851      39        1  1.488983
16    -0.003302  0.018600      40        1  1.439601, topic_info=        Term          Freq         Total Category  logprob  loglift
24      name  34352.000000  34352.000000  Default  30.0000  30.0000
142     word  26461.000000  26461.000000  Default  29.0000  29.0000
28     first  15656.000000  15656.000000  Default  28.0000  28.0000
59      call  14354.000000  14354.000000  Default  27.0000  27.0000
493       go  12489.000000  12489.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1961   medal    305.462471    306.564990  Topic40  -4.6534   4.2372
5320  fellow    305.074302    306.176821  Topic40  -4.6547   4.2372
4914    holy    290.791356    291.893876  Topic40  -4.7026   4.2370
1077  medium    283.770765    284.873284  Topic40  -4.7271   4.2369
5322    risk    276.962050    278.064569  Topic40  -4.7514   4.2368

[1281 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1508     37  0.995730  ability
4643     24  0.997429     able
2527     17  0.998656   accept
5090     37  0.995991  acronym
3136     14  0.999095      act
...     ...       ...      ...
3052     21  0.997126    wrong
938      24  0.994884     yard
205      16  0.999875     year
1155     27  0.997707   yellow
1482     31  0.999492    young

[1600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[31, 25, 38, 35, 28, 40, 21, 18, 14, 37, 2, 33, 30, 6, 7, 1, 29, 9, 26, 8, 24, 15, 22, 11, 3, 27, 16, 39, 5, 4, 12, 34, 13, 20, 10, 19, 23, 32, 36, 17])